# 🎉 [Day 24](https://adventofcode.com/2019/day/24)

In [1]:
import numpy as np


def parse_bugs(inputs):
    return np.array([[int(c == '#') for c in line]
                     for line in inputs])


def convolve2d(matrix, kernel): ## From day17
    """Basic 2d convolutions"""
    lkw = int(np.ceil((kernel.shape[0] - 1) / 2))
    lkh = int(np.ceil((kernel.shape[1] - 1) / 2))
    conv = np.zeros_like(matrix)
    # Pad 
    padded_matrix = np.pad(matrix, 
                           ((lkw, kernel.shape[0] - lkw),
                            (lkh, kernel.shape[1] - lkh)),
                            'constant')
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            conv[i, j] = np.sum(kernel * padded_matrix[i:i+kernel.shape[0], 
                                                       j:j+kernel.shape[1]])
    return conv


kernel = np.array([[0, 1, 0], [1, 0, 1], [0, 1, 0]], dtype=np.int32)
def game_of_life(bugs):
    """One step of the game of life"""
    global kernel
    counts = convolve2d(bugs, kernel)
    bugs = bugs * (counts == 1) + (1 - bugs) * ((counts == 1) + (counts == 2))
    return bugs


biodiversity_map = np.reshape([2**i for i in range(25)], [5, 5])
def biodiversity_index(bugs):
    global biodiversity_map
    return np.sum(bugs * biodiversity_map)

# Unfortunately biodiversity_index is not layout-unique
# so instead we define unique_index to quickly check for equality
def unique_repr(bugs):
    return ''.join(str(x) for line in bugs for x in line)


def find_cycle(bugs):
    seen = {}
    seen[unique_repr(bugs)] = True
    while 1:
        bugs = game_of_life(bugs)
        x = unique_repr(bugs)
        if x in seen:
            return biodiversity_index(bugs)
        seen[x] = True
        
        
def recursive_game_of_life(bugs, n = 10):
    """Plutonian game of life"""
    global kernel
    levels = {0: bugs}
    
    def add_upper_level():
        nonlocal levels
        # starting from top level, see if we should extend it
        top_level = max(levels.keys())
        nxt_level = np.zeros((5, 5), dtype=np.int32)
        if 0 < np.sum(levels[top_level][0, :]) <= 2:
            nxt_level[1, 2] = 1
        if 0 < np.sum(levels[top_level][-1, :]) <= 2:
            nxt_level[3, 2] = 1
        if 0 < np.sum(levels[top_level][:, 0]) <= 2:
            nxt_level[2, 1] = 1
        if 0 < np.sum(levels[top_level][:, -1]) <= 2:
            nxt_level[2, 3] = 1
        if np.sum(nxt_level) > 0:
            levels[top_level + 1] = nxt_level
    
    def add_inner_level():
        nonlocal levels
        # starting from smallest evel, see if we should extend it
        abyss = min(levels.keys())
        nxt_level = np.zeros((5, 5), dtype=np.int32)
        if levels[abyss][1, 2]:
            nxt_level[0, :] = 1
        if levels[abyss][3, 2]:
            nxt_level[-1, :] = 1
        if levels[abyss][2, 1]:
            nxt_level[:, 0] = 1
        if levels[abyss][2, 3]:
            nxt_level[:, -1] = 1
        if np.sum(nxt_level) > 0:
            levels[abyss - 1] = nxt_level
    
    for _ in range(n):
        previous_level = None
        top_level = max(levels.keys())
        abyss = min(levels.keys())
        # Check whether to expand levels
        add_upper_level()
        add_inner_level()
        
        # Update the lower levels
        for l in range(top_level, abyss - 1, -1):
            # Update the inner entries
            counts = convolve2d(levels[l], kernel).astype(np.int32)
            
            # Add counts at the outer border
            if previous_level is not None:
                counts[0, :] += previous_level[1, 2]
                counts[-1, :] += previous_level[3, 2]
                counts[:, 0] += previous_level[2, 1]
                counts[:, -1] += previous_level[2, 3]
                
            # Add counts at the inner border
            if l > abyss:
                counts[1, 2] += np.sum(levels[l - 1][0, :]) 
                counts[3, 2] += np.sum(levels[l - 1][-1, :]) 
                counts[2, 1] += np.sum(levels[l - 1][:, 0]) 
                counts[2, 3] += np.sum(levels[l - 1][:, -1])
                
            # Update
            previous_level = levels[l]
            levels[l] = (levels[l] * (counts == 1) + 
                         (1 - levels[l]) * ((counts == 1) + (counts == 2)))
            levels[l][2, 2] = 0
    
    # Count number of bugs
    return sum(np.sum(arr) for arr in levels.values()), levels

In [2]:
with open('inputs/day24.txt', 'r') as f:
    inputs = f.read().splitlines()
    bugs = parse_bugs(inputs)
    
print("The biodiversity index of the first redundant layout is", find_cycle(bugs))

The biodiversity index of the first redundant layout is 1113073


In [3]:
%%time
print("After 200 iterations of the Plutonian game of life, there are {} bugs\n".format(
    recursive_game_of_life(bugs, n=200)[0]))

After 200 iterations of the Plutonian game of life, there are 1928 bugs

CPU times: user 14.9 s, sys: 215 ms, total: 15.1 s
Wall time: 15 s
